In [134]:
#=========preparation==========#
import torch as torch
import torch.nn as nn
import torch.optim as optim
import torchvision as torchvision
from torchvision import transforms as transforms
import sys

In [135]:
def get_mnist(batch_size):
    if sys.platform.startswith("win"):
        # 0表示不用额外的进程来加速读取数据
        num_workers = 4
    trainset = torchvision.datasets.MNIST(root="data", train=True, download=True, transform=transforms.ToTensor())
    testset = torchvision.datasets.MNIST(root="data", train=False, download=True, transform=transforms.ToTensor())
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)  # 加载
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)  # 加载
    return trainloader, testloader

In [136]:
#----------MySGD----------#
class MyOptimizer(torch.optim.Optimizer):
    def __init__(self, params, lr, batch_size):
        self.lr = lr
        self.batch_size = batch_size
        self.params = params
        super(MyOptimizer, self).__init__(params, {})

    def step_sgd(self, closure=False):
        for param in self.params:
            param.data -= self.lr*param.grad/self.batch_size

    def step_shgd(self, g_inv, closure=False):
        

Utils
========

In [137]:
def loss_function(V,W,H):
    loss = torch.square(torch.norm(V - W @ H, p='fro'))
    
def vectorize_tensor(original_tensor):
    
    """ vectorize a 2D tensor into a 1D tensor as a way of column priority, ie. a column vector

        MxN tensor -> M*Nx1 vector """
    
    if original_tensor.ndim!=2:
        raise ValueError("The input tensor should be 2D.")
    
    M,N= original_tensor.shape[0],original_tensor.shape[1]
    temp_list = []
    for j in range(N):
        temp_list.append(original_tensor[:,j])
    vectorized_tensor = torch.stack(temp_list).view(-1,1)
    return vectorized_tensor

def get_g_inv(W,H):
    m,n,r = W.shape[0],H.shape[1],W.shape[1]
    w_vec = vectorize_tensor(W)
    h_vec = vectorize_tensor(H)
    g_inv = []
    for i in range(w_vec.shape[0]):
        g_inv.append(w_vec[i])
    for j in range(h_vec.shape[0]):
        g_inv.append(h_vec[j])
                                    


In [138]:
def train(inputs,r,lr,iterations,batch_size):
    W = torch.empty((inputs.shape[0],r),dtype=torch.float32).uniform_(0,1).requires_grad_(True)
    H = torch.empty((r,inputs.shape[1]),dtype=torch.float32).uniform_(0,1).requires_grad_(True)

    parameters = [W,H]
    optimizer = MyOptimizer(parameters,lr,batch_size)

    for i in range(iterations):
        
        # loss = loss_function(inputs,W,H)
        loss = torch.square(torch.norm(inputs - W @ H, p='fro'))

        loss.backward()

        optimizer.step_sgd()

        optimizer.zero_grad()

        
    print("loss:",loss.item())
    print("V:\n",inputs)
    print("W:\n",W)
    print("H:\n",H)
    print("W @ H:\n",W @ H)

In [140]:
if __name__ == '__main__':
    r = 8
    lr = 0.001

    batch_size = 1
    iterations =100000
    trainloader,testloader = get_mnist(batch_size)
    inputs = next(iter(trainloader))[0].view(28,28)

    train(inputs,r,lr,iterations,batch_size)

loss: 0.39430227875709534
V:
 tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.3020, 0.9922, 0.5255, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0471, 0.7490, 0.9882, 0.5216, 0.0000, 0.0000,
         0.0000, 0.0000, 